### Importsetup

In [3]:

###########################################################################
from app.main.extraction.dxf_extraction_coordinator import DxfExtractionCoordinator
from app.main.models.wall import Wall 
from app.main.models.architecture_wall import Architectur_Wall
from app.main.models.point import Point 
from app.main.services.walls_extraction import WallsExtraction
from app.main.services.dxf_extraction_simple import DxfExtractionSimple
from app.main.dtos.request.extraction_request_file import ExtractionRequestFile
from app.main.models.line import Line 

###########################################################################
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon as shapePolygon
from shapely.geometry import Point as shapePoint
from shapely.geometry import LineString
import math
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist
import ezdxf
import uuid
import statistics
import matplotlib.pyplot as plt
import operator

###########################################################################
file_path_simple = '../../media/Einfache_Plaene_Compass/'
file_path_architecture = '../../media/Architektenplaene/'
###########################################################################
file_name_simple = 'R-Bau_D_EG.dxf'
file_name_architecutre = 'R-Bau_A_EG.dxf'
###########################################################################
simple_dxf = ezdxf.readfile(file_path_simple+file_name_simple)
architecture_dxf = ezdxf.readfile(file_path_architecture+file_name_architecutre)
simple_model = simple_dxf.modelspace()
arichtecture_model = architecture_dxf.modelspace()


In [ ]:
# DATA-PREPARATION
################ For Matching
dxf_file = ezdxf.readfile(file_path_simple + file_name_simple)
dxf_extraction_simple = DxfExtractionSimple()
################ For Matching
simple_polygons, polygons_with_virtual_entities, extracted_functionalities = dxf_extraction_simple.get_all_polygons(dxf_file ,file_path_simple + file_name_simple)
############################################### For Scaling
simple_polylines_for_scale = simple_model.query("LWPolyline[layer=='RAUMPOLYGON']")


## Helper-Methods

In [4]:
''' This Method gets ezdxf objects from an layer and an entity.'''
def get_polylines_by_layer(layer, entity):
    query_string = f"{entity}[layer=='{layer}']"
    polylines = arichtecture_model.query(query_string)
    polylines_list = []
    for x in polylines:
        polylines_list.append(x)
    return polylines_list, layer

''' This method creates a list of polylines for matching walls'''
def create_architecture_polygons():
    leichtwand_polylines = arichtecture_model.query("LWPolyline[layer=='A_02_LEICHTWAND']")
    tragwand_polylines = arichtecture_model.query("LWPolyline[layer=='A_01_TRAGWAND']")
    architecture_polylines = []
    for x in leichtwand_polylines:
        architecture_polylines.append(x)
    for x in tragwand_polylines:
        architecture_polylines.append(x)
    return architecture_polylines

''' This method plots the all polylines in an dxf-file. '''
def create_dxf_file(simple_polylines: list, architecture_polylines: list):
    doc = ezdxf.new('R2000')
    msp = doc.modelspace()
    for x in simple_polylines:
        msp.add_lwpolyline(x)
    for x in architecture_polylines:
        msp.add_lwpolyline(x)
    doc.saveas("dataForMatching.dxf")


''' This method creates from an Polyline an list of coordinates. '''
def create_coordinates_list(simple_polylines: list):
    coordinates = []
    for x in simple_polylines:
        for y in x:
            coordinates.append(y)
    return coordinates

''' This method finds each min/max x-value of an coordinates tuple.'''
def filter_coordinates(simple_polylines: list, filter_aim):
    coordinates = create_coordinates_list(simple_polylines) 
    if filter_aim == 'MIN':
        item=min(coordinates, key=lambda n: n[0])
        return item
    if filter_aim == 'MAX':
        item=max(coordinates, key=lambda n: n[0])
        return item

''' This method checks if the plans can be matched or not.'''
def check_scaling(architecture_polygons: list, simple_polygons: list):
    boolhandler = False

    min_value_simple = filter_coordinates(simple_polygons, 'MIN')
    max_value_simple = filter_coordinates(simple_polygons, 'MAX')

    point_min_simple = shapePoint(min_value_simple[0], min_value_simple[1])
    point_max_simple = shapePoint(max_value_simple[0], max_value_simple[1])

    distance_simple = point_min_simple.distance(point_max_simple)

    min_value_architecture = filter_coordinates(architecture_polygons, 'MIN')
    max_value_architecture = filter_coordinates(architecture_polygons, 'MAX')

    point_min_architecture = shapePoint(min_value_architecture[0], min_value_architecture[1])
    point_max_architecture = shapePoint(max_value_architecture[0], max_value_architecture[1])

    distance_architecture = point_min_architecture.distance(point_max_architecture)

    factor = distance_simple/distance_architecture
    #print(factor)
    if( 0.95 <= factor <= 1.05):
        boolhandler = True

    return boolhandler

'''This method creates multiple Linestrings from an Polyline with each two coordinate Points.'''
def create_linestring_from_polyline(polyline):
    linestring_list = []
    xy_coordinates_list = []
    index = 0
    #linestring braucht koordianten in xy-format
    for x in polyline:
        x_value = x[0]
        y_value = x[1]
        coordinate = (x_value, y_value)
        xy_coordinates_list.append(coordinate)
    for index in range(len(xy_coordinates_list)-1):
        line = LineString([xy_coordinates_list[index] , xy_coordinates_list[index+1]])
        linestring_list.append(line)
    return linestring_list

''' This method creates a shapely linestring from an wall object.'''
def create_linestring_from_wall(wall):
    line = LineString([(wall.start_point.x_coordinate, wall.start_point.y_coordinate),(wall.end_point.x_coordinate, wall.end_point.y_coordinate)])
    return line

''' This method gets the wallthickness from an architechture_line object from the total architecture wall list.''' 
def get_wall_thickness(architechture_line, architecture_wall_list):
    wall_thickness = 0.0
    for x in architecture_wall_list:
        if(x.id == architechture_line.id_wall):
            wall_thickness = x.wall_thickness
            return wall_thickness, x

''' This method gets the shortes line (wall) from an polyline from the layer "leichtwand" and "tragwand", which are the wallthickness an calculates the average.'''
def calculate_wallthickness(polyline):
    #print(type(polyline))
    wallthickness = 0.0
    linestring_list = create_linestring_from_polyline(polyline)
    list_avg = []
    #print(len(linestring_list))
    for x in linestring_list:
        #print(x.length)
        if(x.length < 0.22):
            list_avg.append(x.length)
    if(len(list_avg) < 1):
        wallthickness = 0.25
        return wallthickness        
    else:
        wallthickness = statistics.mean(list_avg)
        return wallthickness

''' This method get empirical the average Wallthickness from all polylines from one layer. '''
def calculate_avg_value(polyline):
    list_with_min_lenghts = []

    for x in polyline:
        linestring_list = create_linestring_from_polyline(x)
        lenghts = []
        for y in linestring_list:
            lenghts.append(y.length)
        
        min_value = min(lenghts)
        list_with_min_lenghts.append(min_value)
    return round(statistics.mean(list_with_min_lenghts), 2)

simple_walls = WallsExtraction.generete_walls_new(create_coordinates_list(simple_polylines_for_scale))
architecture_polylines_for_scale = create_architecture_polygons()
create_dxf_file(simple_polylines_for_scale, architecture_polylines_for_scale)
######################################## For Matching
architecture_polylines1, layer1 = get_polylines_by_layer('A_02_LEICHTWAND', 'LWPolyline')
architecture_polylines2, layer2 = get_polylines_by_layer('A_01_TRAGWAND', 'LWPolyline')
#######################################

2581 | endPoint: 0.6222028531699522/-0.7163082033292076)
2021-06-10 20:00:09.381 | DEBUG    | app.main.services.orientation_extraction:calculate_orientation:160 - calculated full angle: 217.19686227319093 - for walll (startPoint: 5.165574672456614/24.43274102032204 | endPoint: 5.024823018490039/24.43274102032204)
2021-06-10 20:00:09.403 | DEBUG    | app.main.services.orientation_extraction:calculate_orientation:160 - calculated full angle: 217.19686227319093 - for walll (startPoint: 5.024823018490039/24.50774102032204 | endPoint: 0.5692860184900326/24.50774102032204)
2021-06-10 20:00:09.416 | DEBUG    | app.main.services.orientation_extraction:calculate_orientation:160 - calculated full angle: 168.53011980323782 - for walll (startPoint: 0.5692860184900326/24.50774102032204 | endPoint: -0.5194585033157324/23.26989896542915)
2021-06-10 20:00:09.459 | DEBUG    | app.main.services.orientation_extraction:calculate_orientation:160 - calculated full angle: 217.19686227319127 - for walll (star

In [5]:
'''
This method gets the polylines from the layer "leichtwand" and "tragwand", creates architecture walls and matches them to an simple walls.
'''
def matching_simple_walls_with_polygons(simple_walls: list, simple_polygons: list, architecture_polylines: list, layer: str):
    
    #Hilfslisten und variablen
    linestrings_architecture = []
    architecture_wall_list = []
    line_list_total = []
    amount_matches = 0
    counter = 0

    if(type(architecture_polylines[0]) is ezdxf.entities.lwpolyline.LWPolyline):
        print("Polylines werden gematched!")
        #loop durch alle architecture polylines
        for polyline in architecture_polylines:
            # uuid für die wand erzeugen
            uuid_wall = uuid.uuid1()
            # aus jeder architecture_polyline wird eine liste aus linestrings generiert
            linestrings = create_linestring_from_polyline(polyline)
            # jede der linestrings werden in ein object line mit der uuid und einem linestring der Wall zugeorndet
            lines = []
            for x in linestrings:
                points = x.coords
                line = Line(uuid_wall, x, points[0], points[1])
                lines.append(line)
                line_list_total.append(line)
                linestrings_architecture.append(x)
            # wanddicke ermitteln
            wall_thickness = calculate_wallthickness(polyline)
            # erzeugen einer Archtiecture_wall
            architecture_Wall = Architectur_Wall(uuid_wall, polyline, wall_thickness, layer, lines, 0)
            # architecture wall in die liste der architecture walls einfügen.
            architecture_wall_list.append(architecture_Wall)
        print("architecture_Walls created!")
        # loop durch alle simple_polygons:
        #print(len(simple_polygons))
        for simple_polygon in simple_polygons:
            walls = simple_polygon.walls
            # loop durch alle Wände des simple polygons:
            for simple_wall in walls:
                # wall in linestring umwandeln
                simple_line = create_linestring_from_wall(simple_wall)
                #schauen ob linestring von wall eine line im line array schneidet
                for x in line_list_total:
                    boolhandler = simple_line.intersects(x.line_string)
                    if(boolhandler):
                        amount_matches += 1
                        # architekturplan attribute der simple wall zuordnen und architecture wall bekommt hier simplewall id
                        wall_thickness_calc, architecture_wall_object = get_wall_thickness(x, architecture_wall_list)
                        if(simple_wall.wall_thickness == None):
                            simple_wall.wall_thickness = wall_thickness_calc
                        simple_wall.id = uuid.uuid1()
                        architecture_wall_object.simple_wall_id = simple_wall.id
                        simple_wall.wall_type = layer
                

    for simple_polygon in simple_polygons:
        walls = simple_polygon.walls
        for simple_wall in walls:
            if(simple_wall.wall_thickness != None):
                #ungleich None wanddicke zugeordnet:
                counter += 1

    print('Von ' + str(amount_matches) + ' Matches konnten ' + str(counter) + ' Wanddicken zugeordnet werden.')
    print('FERTIG__________')           
    return simple_polygons, architecture_wall_list


''' Mock-Method for preparation of integration
'''
def coordinate_matching(simple_walls: list, simple_polygons: list, architecture_polyline_for_scale: list, simple_polyline_for_scale: list, architecture_polylines: list , layer:str):
    matched_simplepolygons = []
    layer
    #simple_walls: list, simple_polygons: list, architecture_polylines: list, layer: str
    # Überprüfen des Matchings auf scale Factor
    boolhandler = check_scaling(architecture_polyline_for_scale, simple_polyline_for_scale)
    #print(boolhandler)
    if(boolhandler):
        #ab hier muss das matching koordiniert werden.
        #Layer leichtwand
        if(layer == 1 ):
            layer_name = 'Leichtwand'
            architecture_polylines1, layer1 = get_polylines_by_layer('A_02_LEICHTWAND', 'LWPolyline')
            simplepolygons, architecture_walls = matching_simple_walls_with_polygons(simple_walls, simple_polygons, architecture_polylines, layer_name)
        #layer Tragwand
        if(layer == 2 ):
            layer_name2 = 'Tragwand'
            architecture_polylines2, layer2 = get_polylines_by_layer('A_01_TRAGWAND', 'LWPolyline')
            simplepolygons, architecture_walls = matching_simple_walls_with_polygons(simple_walls, simple_polygons, architecture_polylines, layer_name2)
    return simplepolygons, architecture_walls

In [6]:
# For Testing

res1, res2 = coordinate_matching(simple_walls, simple_polygons, architecture_polylines_for_scale, simple_polylines_for_scale, architecture_polylines1, 1)

res3, res4 = coordinate_matching(simple_walls, simple_polygons, architecture_polylines_for_scale, simple_polylines_for_scale, architecture_polylines2, 2)
#print(res1[1])
#print(res2[0])

Polylines werden gematched!
architecture_Walls created!
Von 368 Matches konnten 269 Wanddicken zugeordnet werden.
FERTIG__________
Polylines werden gematched!
architecture_Walls created!
Von 878 Matches konnten 904 Wanddicken zugeordnet werden.
FERTIG__________
